#**Машинное обучение ИБ-2024**

#**Домашнее задание 1.**
#Регрессия, KNN, LinearRegression.

В данной домашней работе мы будем строить модели для предсказания цены квартиры в России. Ниже приведено описание некоторых колонок набора данных.

date - дата публикации объявления

price - цена в рублях

level- этаж, на котором находится квартира

levels - количество этажей в квартире

rooms - количество комнат в квартире. Если значение -1, то квартира считается апартаментами.

area - площадь квартиры.

kitchen_area - площадь кухни.

geo_lat - Latitude

geo_lon - Longitude

building_type - материал застройки. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

#Часть 0. Начало работы

Для начала работы с данными импортируем библиотеки, которые понадобятся в данном задании.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

Загрузим библиотеку folium для отображения данных на карте по координатам.

In [ ]:
!pip install folium

Распакуем наши данные из архива.

In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/archive (3).zip"

Archive:  /content/drive/MyDrive/Colab Notebooks/archive (3).zip
replace input_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Загрузим данные из csv файла в датафрейм.

In [ ]:
df = pd.read_csv("/content/input_data.csv", sep=";")
df.head()

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0


Отобразим на карте координаты наших построек.

In [ ]:
import folium
from IPython.display import display

map_df = df.loc[:1000]

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# Список точек с широтой и долготой
lats = map_df['geo_lat'].loc[:1000]
longs = map_df['geo_lon'].loc[:1000]
# Добавляем точки на карту
for point in zip(lats, longs):
    folium.Marker(
        location=[point[0], point[1]]
    ).add_to(m)

display(m)

# Часть 1. Подготовим данные для обработки моделями машинного обучения.

**0.5 Балл**. География наших наблюдений в наборе данных крайне большая. Однако мы знаем, что стоимость квартир в Москве и Санкт-Петербурге намного выше, чем в среднем по России. Давайте сделаем признаки, который показывают, находится ли квартира в 20 килиметрах от центра Москвы или находится ли квартира в 20 килиметрах от центра Санкт-Петербурга.

Создайте два признака is_Moscow и is_Saint_Peterburg. Для нахождения расстояния по координатам используйте функцию haversine_distance.

In [ ]:
def haversine_distance(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    b = 2 * np.arcsin(np.sqrt(a))
    r = 6371
    return b * r
msc = (55.755814, 37.6173)
spb = (59.934280, 30.335099)
df['is_Moscow'] = (haversine_distance(df['geo_lat'], df['geo_lon'], msc[0], msc[1]) <= 20).astype(int)
df['is_Saint_Peterburg'] = (haversine_distance(df['geo_lat'], df['geo_lon'], spb[0], spb[1]) <= 20).astype(int)
df.head()


,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id,is_Moscow,is_Saint_Peterburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0,0,0
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN,0,0
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0,0,0
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN,0,0
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0,0,0


**0.5 Балла**. В нашем наборе данных есть признаки, которые мы теоретически можем использовать, например postal_code, но мы это будем делать в рамках домашней работы очень-очень долго. Поэтому предлагается удалить ненужные признаки из датафрейма.

Удалим geo_lat,	geo_lon,	object_type,	postal_code,	street_id,	id_region,	house_id.

In [ ]:
df = df.drop(columns=['geo_lat', 'geo_lon', 'object_type', 'postal_code', 'street_id', 'id_region', 'house_id'])
df.head()

,date,price,level,levels,rooms,area,kitchen_area,building_type,is_Moscow,is_Saint_Peterburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,0,0,0
1,2021-01-01,1450000,5,5,1,33.0,6.0,0,0,0
2,2021-01-01,10700000,4,13,3,85.0,12.0,3,0,0
3,2021-01-01,3100000,3,5,3,82.0,9.0,0,0,0
4,2021-01-01,2500000,2,3,1,30.0,9.0,3,0,0


**0.5 Балл**. Для начала Вам предлагается проанализировать Ваши оставшиеся признаки (колонки) в наборе данных. Какие колонки категориальные? Какие числовые?

Категориальные: (Ваш ответ)

Числовые: (Ваш ответ)

Давайте закодируем категориальные признаки с помощью OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

**0.5 Балл**. Поработаем с числовыми признаками:


1.   Добавьте в ваш датасет два признака: количество дней со дня первого наблюдения (разница между датами объявлений). Возможно, для предсказания цены не так важен этаж, как важно отношение этажа квартиры на количество этажей в доме, добавьте этот признак. После добавления нового признака колонку date можно удалить.
2.   Числовые признаки могут иметь разные порядки. Давайте отнормируем числовые признаки с помощью StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.



**2 Балла**. Реализуйте класс KNNRegressor, который должен делать регрессию методом k ближайших соседей.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
class KNN:
    def __init__(self, k=3, metric='euclidean'):
        self.k = k
        self.metric = metric
    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)
    def predict(self, X):
        X = np.array(X)
        predictions = []
        for x in X:
            distances = self._compute_distances(x)
            k_indices = np.argsort(distances)[:self.k]
            k_nearest_labels = self.y_train[k_indices]
            predictions.append(np.mean(k_nearest_labels))
        return np.array(predictions)
    def _compute_distances(self, x):
        if self.metric == 'euclidean':
            return np.linalg.norm(self.X_train - x, axis=1)
        elif self.metric == 'manhattan':
            return np.sum(np.abs(self.X_train - x), axis=1)
        else:
            raise ValueError("Unknown metric: {}".format(self.metric))
np.random.seed(42)
X = np.random.rand(100, 1)
y = 3 * X.squeeze() + 2 + np.random.randn(100) * 0.5
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
knn = KNN(k=3, metric='euclidean')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("Предсказанные значения:", y_pred)
print("Истинные значения:", y_test)
print(f"Среднеквадратичная ошибка (MSE): {mse}")
print(f"Средняя абсолютная ошибка (MAE): {mae}")
print(f"Корень из среднеквадратичной ошибки (RMSE): {rmse}")


Предсказанные значения: [2.53024039 5.0157194  4.02335167 3.85522023 2.58382659 3.24558629
 2.55338787 4.79189692 2.38890563 2.9927102  3.24558629 3.39496935
 4.60127137 4.50282408 2.35881873 2.27152671 4.02335167 2.13728298
 4.60127137 3.09278029]
Истинные значения: [2.51895186 4.43274422 3.69825895 3.53287313 3.17185592 3.47123115
 2.87899067 5.52219753 2.51945454 3.16714389 3.09978098 4.76572751
 4.59487662 4.80943365 2.76981282 2.35821998 3.97379868 2.3382608
 4.23244782 2.59886128]
Среднеквадратичная ошибка (MSE): 0.21133272492399965
Средняя абсолютная ошибка (MAE): 0.34283961314943356
Корень из среднеквадратичной ошибки (RMSE): 0.4597093918161773


**3 Балла**. Реализуйте класс LinearRegression, поддерживающий обучение градиентными спусками SGD, Momentum, AdaGrad. Используйте градиент для оптимизации функции потерь MSE.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
class LinearRegression:
    def __init__(self, learning_rate=0.01, optimization='SGD', epsilon=1e-8, decay_rate=0.9, max_iter=1000):
        self.learning_rate = learning_rate
        self.optimization = optimization
        self.epsilon = epsilon
        self.decay_rate = decay_rate
        self.max_iter = max_iter
        self.weights = None
        self.bias = None
        self.velocity = None
        self.gradient_squared = None
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        self.velocity = np.zeros(n_features)
        self.gradient_squared = np.zeros(n_features)
        for _ in range(self.max_iter):
            y_pred = np.dot(X, self.weights) + self.bias
            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / n_samples) * np.sum(y_pred - y)
            if self.optimization == 'SGD':
                self.weights -= self.learning_rate * dw
                self.bias -= self.learning_rate * db
            elif self.optimization == 'Momentum':
                self.velocity = self.decay_rate * self.velocity + self.learning_rate * dw
                self.weights -= self.velocity
                self.bias -= self.learning_rate * db
            elif self.optimization == 'AdaGrad':
                self.gradient_squared += dw ** 2
                adjusted_lr = self.learning_rate / (np.sqrt(self.gradient_squared) + self.epsilon)
                self.weights -= adjusted_lr * dw
                self.bias -= adjusted_lr * db
    def predict(self, X):
        return np.dot(X, self.weights) + self.bias
np.random.seed(42)
X = np.random.rand(100, 1)
y = 3 * X.squeeze() + 2 + np.random.randn(100) * 0.5
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
linear_model = LinearRegression(learning_rate=0.01, optimization='SGD', max_iter=1000)
linear_model.fit(X_train, y_train)
y_pred = linear_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("Предсказанные значения:", y_pred)
print("Истинные значения:", y_test)
print(f"Среднеквадратичная ошибка (MSE): {mse}")
print(f"Средняя абсолютная ошибка (MAE): {mae}")
print(f"Корень из среднеквадратичной ошибки (RMSE): {rmse}")


Предсказанные значения: [2.55943727 4.2993548  4.04277891 3.81312031 2.96805294 3.34768121
 3.03788806 4.2329539  2.46948929 3.21034886 3.33050226 3.69804782
 4.1332351  4.41250324 2.67672522 2.75296461 4.04073936 2.581386
 4.16877824 2.78332583]
Истинные значения: [2.51895186 4.43274422 3.69825895 3.53287313 3.17185592 3.47123115
 2.87899067 5.52219753 2.51945454 3.16714389 3.09978098 4.76572751
 4.59487662 4.80943365 2.76981282 2.35821998 3.97379868 2.3382608
 4.23244782 2.59886128]
Среднеквадратичная ошибка (MSE): 0.1897612338995342
Средняя абсолютная ошибка (MAE): 0.29351556556079134
Корень из среднеквадратичной ошибки (RMSE): 0.4356159247542888


# Часть 2. Эксперименты с моделями машинного обучения.

**3 Балла**. Проведите эксперименты с написанными Вами методами машинного обучения. Выделите обучающую и тестовую выборки в отношении 0,8 и 0,2 соответственно. Измерьте ошибку MSE, MAE, RMSE. Заиспользуйте методы KNNRegressor и LinearRegression из библиотеки sklearn, сравните качество Ваших решений и библиотечных.